# Trip Planner Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/trip_planner_agents.ipynb)

## Dependencies

In [4]:
# Install dependencies without output
%pip install langchain_community > /dev/null
%pip install praisonai[crewai] > /dev/null
%pip install requests > /dev/null
%pip install duckduckgo_search > /dev/null

## Tools

In [ ]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from praisonai_tools import BaseTool
from langchain.tools import tool
import ast
import operator as op

class InternetSearchTool(BaseTool):
    name: str = "InternetSearchTool"
    description: str = "Search Internet for relevant information based on a query or latest news"

    def _run(self, query: str):
        ddgs = DDGS()
        results = ddgs.text(keywords=query, region='wt-wt', safesearch='moderate', max_results=5)
        return results


class CalculatorTools():
    @tool("Make a calculation")
    def calculate(operation):
        """Useful to perform any mathematical calculations,
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        try:
            return eval(operation)
        except SyntaxError:
            return "Error: Invalid syntax in mathematical expression"


class WebContentReaderTool(BaseTool):
    name: str = "WebContentReaderTool"
    description: str = "Fetches and reads the main text content from a specified webpage URL."

    def _run(self, url: str) -> str:
        """Reads the content of a webpage and returns up to 5000 characters of text."""
        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract and clean the text content
            text_content = soup.get_text(separator="\n", strip=True)
            return text_content[:5000]  # Limit content to 5000 characters for brevity
        except requests.exceptions.RequestException as e:
            return f"Failed to retrieve content from {url}: {e}"


## YAML Prompt

In [7]:
agent_yaml = """
framework: "crewai"
topic: "Trip Planning and Itinerary Development"
roles:
  city_selection_expert:
    role: "City Selection Expert"
    backstory: |
      An expert in analyzing travel data to pick ideal destinations.
    goal: |
      Select the best city based on weather, seasonal events, and travel costs.
    verbose: true
    allow_delegation: false
    tools:
      - "InternetSearchTool"
      - "WebContentReaderTool"
    tasks:
      identify_city:
        description: |
          Analyze and select the best city for the trip based on specific criteria such as weather patterns, seasonal events, and travel costs. This task involves comparing multiple cities, considering factors like current weather conditions, upcoming cultural or seasonal events, and overall travel expenses. Your final answer must be a detailed report on the chosen city, and everything you found out about it, including the actual flight costs, weather forecast, and attractions.
        expected_output: "Detailed report on the chosen city including flight costs, weather forecast, and attractions."
  local_expert:
    role: "Local Expert at this City"
    backstory: |
      A knowledgeable local guide with extensive information about the city, its attractions, and customs.
    goal: |
      Provide the BEST insights about the selected city.
    verbose: true
    allow_delegation: false
    tools:
      - "InternetSearchTool"
      - "WebContentReaderTool"
    tasks:
      gather_information:
        description: |
          As a local expert on this city, you must compile an in-depth guide for someone traveling there and wanting to have THE BEST trip ever! Gather information about key attractions, local customs, special events, and daily activity recommendations. Find the best spots to go to, the kind of places only a local would know. This guide should provide a thorough overview of what the city has to offer, including hidden gems, cultural hotspots, must-visit landmarks, weather forecasts, and high-level costs. The final answer must be a comprehensive city guide, rich in cultural insights and practical tips, tailored to enhance the travel experience.
        expected_output: "Comprehensive city guide including hidden gems, cultural hotspots, and practical travel tips."
  travel_concierge:
    role: "Amazing Travel Concierge"
    backstory: |
      Specialist in travel planning and logistics with decades of experience.
    goal: |
      Create the most amazing travel itineraries with budget and packing suggestions for the city.
    verbose: true
    allow_delegation: false
    tools:
      - "InternetSearchTool"
      - "WebContentReaderTool"
      - "CalculatorTools"
    tasks:
      plan_itinerary:
        description: |
          Expand this guide into a full 7-day travel itinerary with detailed per-day plans, including weather forecasts, places to eat, packing suggestions, and a budget breakdown.

          You MUST suggest actual places to visit, actual hotels to stay, and actual restaurants to go to.

          This itinerary should cover all aspects of the trip, from arrival to departure, integrating the city guide information with practical travel logistics.

          Your final answer MUST be a complete expanded travel plan, formatted as markdown, encompassing a daily schedule, anticipated weather conditions, recommended clothing and items to pack, and a detailed budget, ensuring THE BEST TRIP EVER. Be specific and give it a reason why you picked each place, what makes them special!
        expected_output: "Complete expanded travel plan with daily schedule, weather conditions, packing suggestions, and budget breakdown."
dependencies: []
"""

## Main

In [12]:
import os
from praisonai import PraisonAI
from google.colab import userdata

# Create a PraisonAI instance with the agent_yaml content
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[InternetSearchTool, WebContentReaderTool, CalculatorTools])

# Add OPENAI_API_KEY Secrets to Google Colab on the Left Hand Side 🔑 or Enter Manually Below
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Run PraisonAI
result = praisonai.run()

# Print the result
print(result) # 10/10


 [2024-11-03 05:02:59][DEBUG]: == Working Agent: City Selection Expert
 [2024-11-03 05:02:59][INFO]: == Starting Task: Analyze and select the best city for the trip based on specific criteria such as weather patterns, seasonal events, and travel costs. This task involves comparing multiple cities, considering factors like current weather conditions, upcoming cultural or seasonal events, and overall travel expenses. Your final answer must be a detailed report on the chosen city, and everything you found out about it, including the actual flight costs, weather forecast, and attractions.



> Entering new CrewAgentExecutor chain...
I need to analyze and select the best city for a trip based on weather patterns, seasonal events, and travel costs. To begin, I'll search for cities that are popular for travel and then evaluate their current weather, upcoming events, and travel costs.

Action: InternetSearchTool  
Action Input: {"query": "best travel destinations October 2023"}  

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': '15 Best Places to Visit in October | U.S. News Travel', 'href': 'https://travel.usnews.com/rankings/best-places-to-visit-in-october/', 'body': 'Acadia National Park. #15 in Best Places to Visit in October 2024. The most beautiful time to visit Acadia National Park is October, hands down. By mid-month, this top attraction in Maine is ...'}, {'title': 'The best places to visit in October 2024 - Lonely Planet', 'href': 'https://www.lonelyplanet.com/articles/where-to-go-in-october', 'body': 'Visit in October, between sweltering summer and chilly, damp winter, to explore (or, more likely, get hopelessly lost in) the tangled skein of 9400 alleys in Fes el-Bali. This oldest section of the walled Medina is romantic and relentless in equal measure. You can admire glorious Moorish zellij tilework and ornately carved wood and alabaster ...'}, {'title': 'The 11 best places to travel in October 2024 - Time Out', 'href': 'https://www.timeout.com/travel/best-places-to-travel-in-october

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': 'Kyoto-shi, Kyoto, Japan Monthly Weather | AccuWeather', 'href': 'https://www.accuweather.com/en/jp/kyoto-shi/224436/october-weather/224436', 'body': 'Get the monthly weather forecast for Kyoto-shi, Kyoto, Japan, including daily high/low, historical averages, to help you plan ahead.'}, {'title': 'Weather in Kyoto in October 2023 (Kyōto-fu) - Detailed Weather Forecast ...', 'href': 'https://world-weather.info/forecast/japan/kyoto/october-2023/', 'body': 'Hourly Week 10-Day 14-Day 30-Day Year. Detailed ⚡ Kyoto Weather Forecast for October 2023 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}, {'title': 'Kyoto, Kyoto, Japan Monthly Weather | AccuWeather', 'href': 'https://www.accuweather.com/en/jp/kyoto-shi/2-224436_1_al/october-weather/2-224436_1_al', 'body': 'Get the monthly weather forecast for Kyoto, Kyoto, Japan, including daily high/low, historical averages, to help you plan ahead.'}, {'title': 'Kyoto October 2023 Historical Weather Data (Japan) - We

...

 

[{'title': '12 Best Places to Stay in Kyoto in the Fall: Dreamy Hotels & Ryokan for ...', 'href': 'https://livejapan.com/en/in-kansai/in-pref-kyoto/in-kyoto-suburbs/article-a2000680/', 'body': "What truly distinguishes NOHGA HOTEL is its commitment to supporting Kyoto's traditions, whether this be food, art, music, or the art of mindfulness. The hotel itself is adorned with an array of captivating artworks, creating a dynamic and inspiring environment for guests. There is also a Meditation room where guests can join a 30-minute session to feel the seasons, days, and to look within."}, {'title': '12 Best Hotels in Kyoto (2024) | Condé Nast Traveler', 'href': 'https://www.cntraveler.com/gallery/best-hotels-in-kyoto', 'body': "Here are 12 of the best hotels in Kyoto, ... 2023, 2024. It's a bold move to open a design-forward property in a city that has fully tied its identity to the past. Even bolder to build it on the ..."}, {'title': 'Top Hotels in Kyoto for October 2023 | Fora Travel

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': 'The 20 Best Restaurants in Kyoto - Bon Traveler', 'href': 'https://www.bontraveler.com/20-best-restaurants-to-eat-at-in-kyoto/', 'body': '2. Honke Daiichiashi. While in Kyoto, this hole-in-the-spot serves ramen to practically only local clientele. Honke Daiichiashi is the spot to be if you want to get a taste of daily culture. The tempura here is also very popular. Address: 845 Higashishiokoji Mukaihatacho, Shimogyo Ward, Kyoto, 600-8213, Japan. 3.'}, {'title': 'THE 10 BEST Restaurants in Kyoto (Updated October 2024)', 'href': 'https://www.tripadvisor.com/Restaurants-g298564-Kyoto_Kyoto_Prefecture_Kinki.html', 'body': '9. Teppanyaki Niyama Kiyamachi. 900 reviews Closed now. Steakhouse, Grill $$ - $$$ Menu. For an appetizer we had the salmon carpaccio and a salad that were both very... Awesome and reasonably priced Kobe beef. 10. Kyoto styled Okonomiyaki & Sake Oagari. 289 reviews.'}, {'title': 'THE 10 BEST Restaurants in Kyoto - Updated October 2024 - Tripadvisor', 'href

### Task Output ###
# 7-Day Travel Itinerary for Kyoto, Japan (October 2023)

Welcome to your amazing 7-day itinerary for Kyoto! This plan will guide you through the best attractions, dining 
options, packing suggestions, and a budget breakdown to ensure you have the best trip ever!

## Overview
- **Travel Dates**: October 1 - October 7, 2023
- **Weather**: Mild temperatures averaging 15-22°C (59-72°F). Expect pleasant days and cooler evenings with a 
chance of rain.
- **Packing Suggestions**: 
  - Layers (light sweaters, long-sleeve shirts)
  - Comfortable walking shoes
  - Umbrella or rain jacket
  - Power adapter (Type A/B)
  - Reusable water bottle

## Day 1: Arrival in Kyoto
- **Morning**: Arrive at Kansai International Airport. Take the Haruka Express train to Kyoto Station (approx. 75 
minutes).
- **Accommodation**: Check-in at **Nijo Castle Hotel** - a perfect blend of modern comfort and traditional Japanese
hospitality.  
  *Cost: Approx. $150/night*
- **Afternoon**: Lunch at **Nishiki Market** - Sample local delicacies like yudofu (tofu hot pot) and matcha 
sweets.
- **Evening**: Visit **Gion District** to explore the traditional wooden machiya houses and catch a glimpse of 
geishas.
- **Dinner**: Dine at **Gion Karyo** for a traditional kaiseki meal.  
  *Cost: Approx. $80 per person*

## Day 2: Temples and Shrines
- **Morning**: Visit **Fushimi Inari Taisha**, famous for its thousands of vermilion torii gates. Hike to the top 
for stunning views.
- **Lunch**: Enjoy a casual meal at **Inari Sushi Koji** near the shrine.  
  *Cost: Approx. $15 per person*
- **Afternoon**: Explore **Kiyomizu-dera** temple. Don’t miss the wooden stage and breathtaking city views.
- **Evening**: Stroll through the charming streets of **Higashiyama**.
- **Dinner**: Try **Okonomiyaki Kizuna** for Kyoto-style savory pancakes.  
  *Cost: Approx. $20 per person*

## Day 3: Historical Exploration
- **Morning**: Visit **Nijo Castle** and explore its beautiful gardens.
- **Lunch**: Eat at **Honke Daiichiashi**, known for its delicious ramen.  
  *Cost: Approx. $12 per person*
- **Afternoon**: Head to **Kinkaku-ji (Golden Pavilion)** to admire the stunning gold-leafed temple.
- **Evening**: Relax at **Kyoto Botanical Gardens** for a peaceful evening stroll.
- **Dinner**: Dine at **Teppanyaki Niyama Kiyamachi** for a memorable grill experience.  
  *Cost: Approx. $50 per person*

## Day 4: Nature and Culture
- **Morning**: Take a trip to **Arashiyama Bamboo Grove**. Enjoy the serene atmosphere and take photos.
- **Lunch**: Have lunch at **Arashiyama Yoshimura**, famous for soba noodles.  
  *Cost: Approx. $25 per person*
- **Afternoon**: Visit **Iwatayama Monkey Park** and interact with the wild monkeys.
- **Evening**: Enjoy a riverboat ride along the Hozu River, surrounded by autumn foliage.
- **Dinner**: Try **Tenzan** for fresh tempura dishes.  
  *Cost: Approx. $30 per person*

## Day 5: Day Trip and Festivals
- **Morning**: Take a day trip to **Kurama** for the **Kurama Fire Festival** (October 22). Enjoy the festive 
atmosphere and traditional performances.
- **Lunch**: Grab a bite in Kurama at **Yudofu Sato** known for its yudofu hot pot.  
  *Cost: Approx. $20 per person*
- **Evening**: Return to Kyoto and have dinner at **Premium Pound Gion** to savor Kobe beef.  
  *Cost: Approx. $70 per person*

## Day 6: Hidden Gems
- **Morning**: Visit the **Rokkakudo Temple**, a serene spot and birthplace of Ikebana (flower arranging).
- **Lunch**: Enjoy a light lunch at **Cafe Bibliotic Hello!** for a cozy atmosphere.  
  *Cost: Approx. $15 per person*
- **Afternoon**: Explore **Nonomiya Shrine**, known for its moss garden and peaceful ambiance.
- **Evening**: Visit the **Kyoto Antique Fair** (October 27-29) for unique treasures.
- **Dinner**: Dine at **Gion Nanba**, a Michelin-starred kaiseki restaurant.  
  *Cost: Approx. $100 per person*

## Day 7: Last Day and Departure
- **Morning**: Visit **Kyoto Tower** for panoramic views of the city.
- **L

None
